# Testing Cosine Similarity Loss

In [7]:
import torch
import torch.nn.functional as F
import numpy as np

In [2]:
# batch_size, num_vectors, num_dims
dummy_shape = (2,3,3)

In [5]:
x_1 = torch.randn(dummy_shape)
x_2 = torch.randn(dummy_shape)

In [6]:
x_1, x_2

(tensor([[[-0.4128,  1.9865,  0.7096],
          [ 0.1461,  0.3724, -0.8140],
          [-0.2208, -0.8880,  0.7061]],
 
         [[-1.6274, -1.2021, -0.7152],
          [ 0.6671,  1.7572,  0.0586],
          [-0.6044,  2.2627, -0.6261]]]),
 tensor([[[-2.0277,  0.0643, -0.8230],
          [ 2.4958,  2.5395, -0.7325],
          [-0.9439, -0.8864, -2.0320]],
 
         [[ 0.1137, -0.0143, -1.6874],
          [ 0.9421,  0.3634,  1.4555],
          [-2.0639,  0.6597, -0.1250]]]))

## Non-vectorized Approach

In [16]:
result = np.zeros((dummy_shape[0], dummy_shape[1], dummy_shape[1]))

for i in range(dummy_shape[0]):
    for j in range(dummy_shape[1]):
        for k in range(dummy_shape[1]):
            result[i][j][k] = F.cosine_similarity(x_1[i][j], x_2[i][k], dim=0)

In [10]:
F.cosine_similarity(x_1[0][0], x_2[0][0])

tensor(0.0809)

In [18]:
# cosine similarity of x_1[0][0] against all vectors in x_2[0]
result[0][0]

array([ 0.08090946,  0.44723642, -0.54316366])

In [22]:
result = np.zeros((dummy_shape[0], dummy_shape[1], dummy_shape[1]))

for i in range(dummy_shape[0]):
    for j in range(dummy_shape[1]):
        for k in range(dummy_shape[1]):
            result[i][j][k] = F.cosine_similarity(x_1[i][j], x_1[i][k], dim=0)

In [23]:
result[0]

array([[ 1.        ,  0.05220373, -0.47163579],
       [ 0.05220373,  1.        , -0.89451832],
       [-0.47163579, -0.89451832,  1.        ]])

In [24]:
result[1]

array([[ 1.        , -0.80283338, -0.24766201],
       [-0.80283338,  1.        ,  0.77568722],
       [-0.24766201,  0.77568722,  1.        ]])

## Testing custom function

In [30]:
def cosine_pairwise(x):
    # https://github.com/pytorch/pytorch/issues/11202#issuecomment-619532801
    x = x.permute((1, 2, 0))
    cos_sim_pairwise = F.cosine_similarity(x, x.unsqueeze(1), dim=-2)
    cos_sim_pairwise = cos_sim_pairwise.permute((2, 0, 1))
    return cos_sim_pairwise

In [31]:
cosine_pairwise(x_1)[0]

tensor([[ 1.0000,  0.0522, -0.4716],
        [ 0.0522,  1.0000, -0.8945],
        [-0.4716, -0.8945,  1.0000]])

In [32]:
cosine_pairwise(x_1)[1]

tensor([[ 1.0000, -0.8028, -0.2477],
        [-0.8028,  1.0000,  0.7757],
        [-0.2477,  0.7757,  1.0000]])

In [35]:
x_1

tensor([[[-0.4128,  1.9865,  0.7096],
         [ 0.1461,  0.3724, -0.8140],
         [-0.2208, -0.8880,  0.7061]],

        [[-1.6274, -1.2021, -0.7152],
         [ 0.6671,  1.7572,  0.0586],
         [-0.6044,  2.2627, -0.6261]]])

In [39]:
x_1[:, :, torch.randperm(3)[:2]]

tensor([[[-0.4128,  1.9865],
         [ 0.1461,  0.3724],
         [-0.2208, -0.8880]],

        [[-1.6274, -1.2021],
         [ 0.6671,  1.7572],
         [-0.6044,  2.2627]]])

In [38]:
torch.randperm(3)[:2]

tensor([2, 1])